# ทดสอบโมเดล Hand B - คู่มือสำหรับมือใหม่

**สำหรับ Person 2: ผู้ดูแลโมเดล Hand B**

## เกี่ยวกับโมเดลนี้

โมเดลนี้จะจดจำท่าภาษามือไทย **9 คำ**:
- **ขอบคุณ** (มือแตะหน้าผากแล้วเหวี่ยงออก)
- **ไม่เป็นไร** (มือโบกซ้ายขวา)
- **สบายดี** (มือชูนิ้วโป้ง)
- **โชคดี** (มือกำปั้นชูขึ้น)
- **เก่ง** (นิ้วโป้งชูขึ้น)
- **อิ่ม** (มือถูท้องเป็นวงกลม)
- **หิว** (มือแตะท้อง)
- **เศร้า** (มือปิดหน้าหรือเช็ดน้ำตา)
- **idle** (มือพักธรรมชาติ)

## วิธีใช้ Notebook นี้

1. รันทุกเซลล์ตามลำดับ
2. ตรวจสอบว่าโมเดลทำงานได้
3. ทดสอบกับรูปภาพจริง
4. บันทึกผลการทดสอบ

In [ ]:
# ขั้นตอนที่ 1: ติดตั้ง Libraries

print("+ กำลังติดตั้ง libraries สำหรับ Hand B...")
print("  กรุณารอสักครู่ (อาจใช้เวลา 1-2 นาที)")

# ติดตั้ง libraries ที่จำเป็น
import sys
!{sys.executable} -m pip install tensorflow opencv-python pillow numpy matplotlib

print("+ ติดตั้งเสร็จเรียบร้อย!")
print("="*50)

In [ ]:
# ขั้นตอนที่ 2: Import Libraries

print("+ กำลัง import libraries...")

# Import libraries ที่จำเป็นทั้งหมด
import tensorflow as tf
import json
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import os

# แสดงข้อมูลระบบ
print("+ Import สำเร็จ!")
print(f"  - TensorFlow Version: {tf.__version__}")
print(f"  - Current Directory: {os.getcwd()}")
print("="*50)

In [ ]:
# ขั้นตอนที่ 3: ตรวจสอบไฟล์โมเดล

print("+ ตรวจสอบไฟล์โมเดล Hand B...")

# ไฟล์ที่จำเป็นสำหรับโมเดล
required_files = ['model.json', 'metadata.json', 'weights.bin']

print("  ผลการตรวจสอบ:")
all_files_found = True

for file in required_files:
    if os.path.exists(file):
        file_size = os.path.getsize(file) / 1024  # KB
        print(f"  + {file} - พบแล้ว ({file_size:.1f} KB)")
    else:
        print(f"  - {file} - ไม่พบ!")
        all_files_found = False

if all_files_found:
    print("\n+ ไฟล์โมเดลพร้อมใช้งาน!")
else:
    print("\n- ไฟล์ไม่ครบ! โปรดวางไฟล์โมเดลในโฟลเดอร์นี้")
    print("  ไฟล์ที่ต้องมี: model.json, metadata.json, weights.bin")

print("="*50)

In [ ]:
# ขั้นตอนที่ 4: สร้าง HandB Model Class

print("+ สร้าง HandB Model Class...")

class HandBModel:
    """
    คลาสง่ายๆ สำหรับโหลดและใช้งานโมเดล Hand B
    """
    
    def __init__(self):
        """เริ่มต้น HandB Model"""
        print("  o กำลังสร้าง HandB Model...")
        self.model = None
        self.class_names = []
        print("  + สร้าง Instance เรียบร้อย!")
        
    def load_model(self):
        """โหลดโมเดลจากไฟล์"""
        print("  o กำลังโหลดโมเดล...")
        
        try:
            # อ่าน metadata เพื่อดูคลาสต่างๆ
            with open('metadata.json', 'r', encoding='utf-8') as f:
                metadata = json.load(f)
                self.class_names = [item['className'] for item in metadata['labels']]
            
            # โหลดโมเดล TensorFlow
            self.model = tf.keras.models.load_model('model.json')
            
            print("  + โหลดโมเดลสำเร็จ!")
            print(f"    จำนวนคลาส: {len(self.class_names)}")
            
            return True
            
        except Exception as e:
            print(f"  - เกิดข้อผิดพลาด: {e}")
            return False
    
    def predict_simple(self, image_path):
        """ทำนายแบบง่าย - ส่งกลับผลลัพธ์ที่เข้าใจง่าย"""
        if self.model is None:
            return "โปรดโหลดโมเดลก่อน"
            
        try:
            # เตรียมรูปภาพ
            img = Image.open(image_path).convert('RGB')
            img = img.resize((224, 224))
            img_array = np.array(img) / 255.0
            img_array = np.expand_dims(img_array, axis=0)
            
            # ทำนาย
            predictions = self.model.predict(img_array, verbose=0)[0]
            
            # หาผลลัพธ์ที่ดีที่สุด
            best_index = np.argmax(predictions)
            best_class = self.class_names[best_index]
            confidence = predictions[best_index] * 100
            
            return {
                'prediction': best_class,
                'confidence': confidence,
                'success': True
            }
            
        except Exception as e:
            return {
                'error': str(e),
                'success': False
            }

print("+ HandB Model Class พร้อมใช้งาน!")
print("="*50)

In [ ]:
# ขั้นตอนที่ 5: โหลดโมเดล Hand B

print("+ เริ่มโหลดโมเดล Hand B...")

# สร้าง instance ของโมเดล
handB_model = HandBModel()

# พยายามโหลดโมเดล
success = handB_model.load_model()

if success:
    print("+ โมเดลพร้อมใช้งาน!")
    print("  คลาสที่โมเดลจำได้:")
    for i, name in enumerate(handB_model.class_names, 1):
        print(f"    {i}. {name}")
else:
    print("- ไม่สามารถโหลดโมเดลได้")
    print("  ตรวจสอบว่าไฟล์ model.json และ metadata.json อยู่ในโฟลเดอร์")

print("="*50)

In [ ]:
# ขั้นตอนที่ 6: ทดสอบด้วยรูปภาพ (แบบง่าย)

def test_image_simple(image_name="test_image.jpg"):
    """
    ฟังก์ชันง่ายๆ สำหรับทดสอบโมเดล
    """
    
    print(f"+ กำลังทดสอบไฟล์: {image_name}")
    
    # ตรวจสอบว่ามีไฟล์รูปหรือไม่
    if not os.path.exists(image_name):
        print(f"- ไม่พบไฟล์ {image_name}")
        print("  วางไฟล์รูปในโฟลเดอร์เดียวกับ notebook นี้")
        return
    
    # แสดงรูปภาพ
    print("  o แสดงรูปภาพ...")
    img = Image.open(image_name)
    plt.figure(figsize=(6, 4))
    plt.imshow(img)
    plt.title(f"ทดสอบ Hand B: {image_name}")
    plt.axis('off')
    plt.show()
    
    # ทำนาย
    print("  o กำลังทำนาย...")
    result = handB_model.predict_simple(image_name)
    
    # แสดงผล
    if result['success']:
        prediction = result['prediction']
        confidence = result['confidence']
        
        print(f"  + ผลการทำนาย: {prediction}")
        print(f"    ความมั่นใจ: {confidence:.1f}%")
        
        if confidence >= 80:
            print("    สถานะ: เยี่ยมมาก! ✓")
        elif confidence >= 60:
            print("    สถานะ: ดี ✓")
        else:
            print("    สถานะ: ต่ำ - ลองถ่ายรูปใหม่")
            
    else:
        print(f"  - ข้อผิดพลาด: {result['error']}")

# ทดสอบทันที
print("+ เริ่มทดสอบโมเดล Hand B...")
test_image_simple("test_image.jpg")

print("\n" + "="*50)
print("o วิธีเพิ่มรูปทดสอบ:")
print("  1. ถ่ายรูปท่าภาษามือ")
print("  2. บันทึกเป็น test_image.jpg")
print("  3. วางในโฟลเดอร์เดียวกับ notebook")
print("  4. รัน cell นี้อีกครั้ง")
print("="*50)

In [ ]:
# ขั้นตอนที่ 7: ทดสอบหลายรูปพร้อมกัน (ขั้นสูง)

def test_multiple_images():
    """
    ทดสอบรูปหลายๆ รูปพร้อมกัน
    """
    
    print("+ กำลังค้นหารูปภาพในโฟลเดอร์...")
    
    # หาไฟล์รูปภาพทั้งหมด
    image_extensions = ['.jpg', '.jpeg', '.png']
    image_files = []
    
    for file in os.listdir('.'):
        if any(file.lower().endswith(ext) for ext in image_extensions):
            image_files.append(file)
    
    if not image_files:
        print("- ไม่พบไฟล์รูปภาพ")
        print("  วางไฟล์ .jpg, .jpeg หรือ .png ในโฟลเดอร์นี้")
        return
    
    print(f"  พบรูปภาพ {len(image_files)} ไฟล์")
    
    # ทดสอบทีละไฟล์
    results = []
    
    for i, img_file in enumerate(image_files, 1):
        print(f"\n  ({i}/{len(image_files)}) ทดสอบ: {img_file}")
        
        result = handB_model.predict_simple(img_file)
        
        if result['success']:
            prediction = result['prediction']
            confidence = result['confidence']
            
            print(f"    ผล: {prediction} ({confidence:.1f}%)")
            
            results.append({
                'file': img_file,
                'prediction': prediction,
                'confidence': confidence
            })
        else:
            print(f"    ข้อผิดพลาด: {result['error']}")
    
    # สรุปผล
    if results:
        print("\n" + "="*40)
        print("+ สรุปผลการทดสอบทั้งหมด:")
        
        for r in results:
            status = "เยี่ยม" if r['confidence'] >= 80 else "ดี" if r['confidence'] >= 60 else "ต่ำ"
            print(f"  {r['file']:15s} -> {r['prediction']:10s} ({r['confidence']:4.1f}%) [{status}]")
        
        # คำนวณค่าเฉลี่ย
        avg_confidence = sum(r['confidence'] for r in results) / len(results)
        print(f"\n  ความมั่นใจเฉลี่ย: {avg_confidence:.1f}%")

# เรียกใช้ฟังก์ชัน
print("+ ฟังก์ชันทดสอบหลายรูปพร้อมแล้ว!")
print("  รันคำสั่ง: test_multiple_images()")
print("="*50)

In [ ]:
# ขั้นตอนที่ 8: ทดสอบด้วยกล้องเว็บแคม (แบบง่าย)

def start_webcam_test():
    """
    ฟังก์ชันง่ายๆ สำหรับทดสอบด้วยกล้อง
    กด 'S' = ถ่ายรูปและทำนาย
    กด 'Q' = ออก
    """
    
    print("+ กำลังเปิดกล้อง...")
    
    try:
        # เปิดกล้อง
        cap = cv2.VideoCapture(0)
        
        if not cap.isOpened():
            print("- ไม่สามารถเปิดกล้องได้!")
            print("  ตรวจสอบการเชื่อมต่อกล้อง")
            return
        
        print("+ กล้องพร้อมใช้งาน!")
        print("  การควบคุม:")
        print("    กด 'S' = ถ่ายรูปและทำนาย")
        print("    กด 'Q' = ออกจากระบบ")
        
        test_count = 0
        
        while True:
            # อ่านภาพจากกล้อง
            ret, frame = cap.read()
            
            if not ret:
                print("- ไม่สามารถอ่านภาพจากกล้องได้")
                break
            
            # แสดงข้อความบนหน้าจอ
            cv2.putText(frame, "Hand B Test - Press S to capture", 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"Tests: {test_count}", 
                       (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
            
            # แสดงกรอบสำหรับวางมือ
            cv2.rectangle(frame, (100, 100), (500, 400), (255, 100, 0), 2)
            cv2.putText(frame, "Put your hand here", 
                       (110, 95), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 100, 0), 2)
            
            cv2.imshow('Hand B Webcam Test', frame)
            
            # รอคำสั่งจากแป้นพิมพ์
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord('s') or key == ord('S'):  # ถ่ายรูปและทำนาย
                test_count += 1
                print(f"\n  การทดสอบครั้งที่ {test_count}:")
                
                # บันทึกรูปภาพชั่วคราว
                temp_image = "temp_capture.jpg"
                cv2.imwrite(temp_image, frame)
                
                # ทำนาย
                result = handB_model.predict_simple(temp_image)
                
                if result['success']:
                    prediction = result['prediction']
                    confidence = result['confidence']
                    
                    print(f"    ผล: {prediction}")
                    print(f"    ความมั่นใจ: {confidence:.1f}%")
                    
                    # แสดงผลบนหน้าจอกล้อง
                    result_text = f"{prediction} ({confidence:.1f}%)"
                    cv2.putText(frame, result_text, 
                               (10, 450), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
                    
                    # แสดงสถานะ
                    if confidence >= 80:
                        status_color = (0, 255, 0)  # เขียว = ดีมาก
                        status_text = "Excellent!"
                        print("    สถานะ: เยี่ยมมาก!")
                    elif confidence >= 60:
                        status_color = (0, 255, 255)  # เหลือง = ดี
                        status_text = "Good!"
                        print("    สถานะ: ดี!")
                    else:
                        status_color = (0, 100, 255)  # ส้ม = ต่ำ
                        status_text = "Try Again"
                        print("    สถานะ: ควรลองใหม่")
                    
                    cv2.putText(frame, status_text, 
                               (10, 480), cv2.FONT_HERSHEY_SIMPLEX, 0.6, status_color, 2)
                    
                    # แสดงผลเป็นเวลา 2 วินาที
                    cv2.imshow('Hand B Webcam Test', frame)
                    cv2.waitKey(2000)
                
                # ลบไฟล์ชั่วคราว
                if os.path.exists(temp_image):
                    os.remove(temp_image)
                    
            elif key == ord('q') or key == ord('Q'):  # ออกจากโปรแกรม
                print("\n  ปิดกล้อง...")
                break
        
        # ปิดกล้องและหน้าต่าง
        cap.release()
        cv2.destroyAllWindows()
        
        print(f"+ เสร็จสิ้นการทดสอบ (ทดสอบทั้งหมด {test_count} ครั้ง)")
        
    except Exception as e:
        print(f"- เกิดข้อผิดพลาด: {e}")
        print("  ลองรีสตาร์ท notebook แล้วลองใหม่")

print("+ ฟังก์ชันทดสอบกล้องพร้อมแล้ว!")
print("  รันคำสั่ง: start_webcam_test()")

print("\n+ ท่าที่ต้องทดสอบสำหรับ Hand B:")
gestures = ["ขอบคุณ", "ไม่เป็นไร", "สบายดี", "โชคดี", "เก่ง", "อิ่ม", "หิว", "เศร้า", "idle"]
for i, gesture in enumerate(gestures, 1):
    print(f"  {i}. {gesture}")

print("="*50)

In [ ]:
# ขั้นตอนที่ 9: สรุปและตรวจสอบโมเดล

def check_model_quality():
    """
    ฟังก์ชันตรวจสอบคุณภาพโมเดลแบบง่าย
    """
    
    print("+ ตรวจสอบคุณภาพโมเดล Hand B...")
    
    if handB_model.model is None:
        print("- โมเดลยังไม่ได้โหลด")
        return
    
    # ตรวจสอบขนาดไฟล์
    model_size = os.path.getsize('model.json') / 1024  # KB
    weights_size = os.path.getsize('weights.bin') / (1024 * 1024)  # MB
    
    print(f"  ขนาดไฟล์โมเดล: {model_size:.1f} KB")
    print(f"  ขนาดไฟล์ weights: {weights_size:.1f} MB")
    
    # ตรวจสอบจำนวนคลาส
    num_classes = len(handB_model.class_names)
    expected_classes = 9  # Hand B ควรมี 9 คลาส
    
    print(f"  จำนวนคลาส: {num_classes}")
    
    if num_classes == expected_classes:
        print("  + จำนวนคลาสถูกต้อง!")
    else:
        print(f"  - จำนวนคลาสไม่ถูกต้อง (ควรเป็น {expected_classes})")
    
    # แสดงคลาสทั้งหมด
    print("  คลาสในโมเดล:")
    for i, name in enumerate(handB_model.class_names, 1):
        print(f"    {i}. {name}")
    
    # คำแนะนำ
    print("\n+ คำแนะนำการใช้งาน:")
    print("  - ทดสอบกับรูปภาพก่อน (ขั้นตอนที่ 6)")
    print("  - ถ้าผลดี ให้ทดสอบด้วยกล้อง (ขั้นตอนที่ 8)")
    print("  - ความมั่นใจ > 60% = ผ่าน")
    print("  - ความมั่นใจ > 80% = ดีมาก")

# รันการตรวจสอบ
check_model_quality()
print("="*50)

## สรุป - การทดสอบโมเดล Hand B

### ✅ สิ่งที่ควรได้จากการทดสอบ:

1. **โมเดลโหลดสำเร็จ** - ไม่มี error
2. **ความมั่นใจ ≥ 60%** - ผลการทำนายน่าเชื่อถือ  
3. **แยกแยะ idle ได้** - ไม่ทำนายผิดเมื่อไม่ทำท่า
4. **ทดสอบครบ 9 ท่า** - ขอบคุณ, ไม่เป็นไร, สบายดี, โชคดี, เก่ง, อิ่ม, หิว, เศร้า, idle

### 🔧 หากผลไม่ดี - วิธีแก้ไข:

**ปัญหา: ความมั่นใจต่ำ (< 60%)**
- เพิ่มรูปภาพในคลาสที่มีปัญหา
- ถ่ายรูปในแสงที่ดี มุมมองชัดเจน
- Train โมเดลใหม่ด้วย Epochs มากขึ้น

**ปัญหา: สับสนระหว่างท่า**  
- เพิ่มรูปเพื่อแยกแยะท่าที่คล้ายกัน
- ทำท่าให้ชัดเจน แตกต่างกันมากขึ้น

**ปัญหา: idle ไม่ถูกต้อง**
- เพิ่มรูป idle ให้มากที่สุด (150+ รูป)
- ถ่าย idle หลากหลายท่าวาง

### 📋 ขั้นตอนถัดไป:

1. **นำไปใช้ใน Frontend** - คัดลอกไฟล์โมเดลไปยัง `public/models/handB/`
2. **ทดสอบในเว็บไซต์** - รันด้วย `npm run dev` 
3. **ประสานทีม** - แจ้ง Person 3 (Frontend) ว่าโมเดลพร้อม
4. **ทดสอบรวม** - ร่วมกับ Hand A และ Face Detection

### 📞 ขอความช่วยเหลือ:

- **Person 1:** เปรียบเทียบผลกับ Hand A
- **Person 3:** ปัญหาการแสดงผลใน Frontend  
- **Person 4:** การเชื่อมต่อกับ Backend API
- **ทีม:** ปัญหาทั่วไปของโปรเจค

---

**🎉 เมื่อโมเดลพร้อมแล้ว: ความมั่นใจ > 60%, ทดสอบครบทุกท่า, ไฟล์โมเดลถูกต้อง**

**ยินดีด้วย! โมเดล Hand B ของคุณพร้อมใช้งาน!**